### sklearn.impute.SimpleImputer
- 결측이 있는 변수의 대표값으로 결측을 대체하는 인스턴스
- 주요 입력
  - strategy: 대표 통계량을 지정('mean', 'most_frequent', 'median')
  - 변수 타입에 따라 두 개의 인스턴스를 같이 적용해야 할 수 있음

## 단순한 케이스: 모든 특징의 타입이 같은 경우

In [1]:
import os
import pandas as pd
os.chdir(r'C:\Users\신상훈\Desktop\preprocessing\part-4.-머신러닝을-위한-필수-전처리\Part 4. 머신러닝을 위한 필수 전처리\데이터')

In [2]:
df = pd.read_csv('cleveland.csv')

In [3]:
# 특징과 라벨 분리
X = df.drop('Output', axis= 1)
Y = df['Output']

In [4]:
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [6]:
# 결측치 확인
Train_X.isnull().sum()
# 결측치가 많지 않음
# 지워도 무방한 수치이지만, 새로 들어온 데이터에 결측이 있을 수도 있다는 도메인 지식이 있다고 가정

Age         0
Sex         0
Cp          0
Trestbps    0
Chol        0
Fbs         0
Restecg     0
Thalach     0
Exang       0
Oldpeak     0
Slope       0
Ca          4
Thal        1
dtype: int64

In [8]:
# 평균 상관 계수 확인 (주의: 모든 변수가 연속형이므로 가능한 접근)
Train_X.corr().sum() / (len(Train_X.columns)-1)
# 수치가 높지 않다고 판단 => 특징 간 관계가 크지 않음 => 대표값 대체 활용 가능 판단

Age         0.162536
Sex         0.126407
Cp          0.084902
Trestbps    0.123191
Chol        0.109878
Fbs         0.128618
Restecg     0.128064
Thalach    -0.082749
Exang       0.138316
Oldpeak     0.179704
Slope       0.148647
Ca          0.144787
Thal        0.191649
dtype: float64

In [9]:
# 대표값을 활용한 결측치 대체
from sklearn.impute import SimpleImputer
# SimpleImputer 인스턴스화
SI = SimpleImputer(strategy = 'mean')
#학습
SI.fit(Train_X)

# sklearn instance의 출력은 ndarray이므로 다시 DataFrame으로 바꿔줌
Train_X = pd.DataFrame(SI.transform(Train_X), columns = Train_X.columns)
Test_X = pd.DataFrame(SI.transform(Test_X), columns = Test_X.columns)

In [10]:
Train_X.isnull().sum()

Age         0
Sex         0
Cp          0
Trestbps    0
Chol        0
Fbs         0
Restecg     0
Thalach     0
Exang       0
Oldpeak     0
Slope       0
Ca          0
Thal        0
dtype: int64

## 복잡한 케이스: 다른 타입의 특징이 있는 경우

In [11]:
df = pd.read_csv('saheart.csv')

In [12]:
# 특징과 라벨 분리
X = df.drop('Chd', axis = 1)
Y = df['Chd']

In [13]:
# 학습 데이터와 평가 데이터로 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [14]:
# 결측치 확인
Train_X.isnull().sum()
# 결측치가 많지 않음

Sbp          0
Tobacco      0
Ldl          0
Adiposity    0
Typea        0
Obesity      9
Alcohol      3
Age          0
Famhist      5
dtype: int64

In [15]:
# 평균 상관 계수 확인 (주의: 모든 변수가 연속형이므로 가능한 접근)
Train_X.corr().sum() / (len(Train_X.columns) - 1)

# 수치가 높지 않다고 판단 => 특징 간 관계가 크지 않음 => 대표값 대체 활용 가능 판단

Sbp          0.282870
Tobacco      0.197851
Ldl          0.281151
Adiposity    0.414110
Typea        0.100179
Obesity      0.333502
Alcohol      0.211788
Age          0.366778
Famhist      0.251342
dtype: float64

In [16]:
# Famhist: 범주형 변수
# 그 외 변수: 연속형 변수
# 대표값을 평균을 사용할지, 최빈값을 사용할지 결정이 어려움 => 둘 다 사용해야 함 # 범주형은 최빈값으로 대체
# 따라서 데이터를 분할해야 함
Train_X_cate = Train_X[['Famhist']]
Train_X_cont = Train_X.drop('Famhist', axis = 1)

Test_X_cate = Test_X[['Famhist']]
Test_X_cont = Test_X.drop('Famhist', axis = 1)


In [17]:
# 대표값을 활용한 결측치 대체
from sklearn.impute import SimpleImputer
# SimpleImputer 인스턴스화
SI_mode = SimpleImputer(strategy = 'most_frequent')
SI_mean = SimpleImputer(strategy = 'mean')

# 학습
SI_mode.fit(Train_X_cate)
SI_mean.fit(Train_X_cont)

# sklearn instance의 출력은 ndarray이므로 다시 DataFrame으로 바꿔줌
Train_X_cate = pd.DataFrame(SI_mode.transform(Train_X_cate),
                            columns = Train_X_cate.columns)

Test_X_cate = pd.DataFrame(SI_mode.transform(Test_X_cate),
                           columns = Test_X_cate.columns)

Train_X_cont = pd.DataFrame(SI_mean.transform(Train_X_cont),
                            columns = Train_X_cont.columns)

Test_X_cont = pd.DataFrame(SI_mean.transform(Test_X_cont),
                           columns = Test_X_cont.columns)

# 다시 두 데이터를 이어붙여야 함
Train_X = pd.concat([Train_X_cate, Train_X_cont], axis = 1)
Test_X = pd.concat([Test_X_cate, Test_X_cont], axis = 1)

In [18]:
Train_X.isnull().sum()

Famhist      0
Sbp          0
Tobacco      0
Ldl          0
Adiposity    0
Typea        0
Obesity      0
Alcohol      0
Age          0
dtype: int64

In [ ]:
# Tip. 이진형 변수와 연속형 변수만 포함된 경우에는 SI_mean만 사용하여 결측치를 평균으로 대체한 뒤에, 
# 이진형 변수에 대해서만 round 처리를 하면 하나의 인스턴스만 활용할 수 있음